In [1]:
import datetime
import pandas as pd
import hashlib
import json
import csv
# from datetime import datetime, date, timedelta
import hashlib
import os
from os.path import exists
from copy import deepcopy


In [2]:
# def get_hash(n, a):
#     return hashlib.sha256(n+"FCA (UK) Unauthorised firms and Individuals List, United Kingdom"+a).lower().encode().hexdigest()

def get_hash(n):
    return hashlib.sha256(((n+"FCA (UK) Unauthorised firms and Individuals List, United Kingdom").lower()).encode()).hexdigest()


last_updated_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")


In [3]:
df = pd.read_excel("UK_FCA_Unauthorised firms and individuals.xlsx")



df.fillna("", inplace=True)


frame_ = df.to_dict(orient="record")

/tmp/ipykernel_2144209/3193290034.py:8: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  frame_ = df.to_dict(orient="record")


In [18]:
df.head()

,Sr No,Firm Type,Type,Name,Alias,Contact Number,Email,Website,Complete_address,City,State,Country,Pin code,Url 1
0,1,Unauthorised,Entity,24FOREX TRADE STOCKS,,17178236772,support@24forextradestocks.com,https://24forextradestocks.com/,66 Great Suffolk Street,London,England,United Kingdom,SE1 0BL,https://www.fca.org.uk/news/warnings/24forex-t...
1,2,Unauthorised,Entity,7dayscrypto Inc,,15134018095,support@7dayscrypto.com,http://7dayscrypto.com/,2690 South Michigan Ave,Chicago,Illinois,\tAmerica,IL 60616,https://www.fca.org.uk/news/warnings/7dayscryp...
2,3,Unauthorised,Entity,365Falcon,,15134018095,support@365falcon.com,https://www.365falcon.com,Rue de la Cite 4,Geneva,Geneva,Switzerland,1204,https://www.fca.org.uk/news/warnings/365falcon
3,4,Unauthorised,Entity,450stocktradefx Ltd,,447404423692,support@450stocktradefx.com,https://450stocktradefx.com/en,,,,,,https://www.fca.org.uk/news/warnings/450stockt...
4,5,Unauthorised,Entity,24FXODDPRO,,,support@24fxoddpro.com,https://24fxoddpro.com/,"Unbolted, 6, Lloyd's Avenue, Aldgate",London,England,United Kingdom,EC3N 3AX,https://www.fca.org.uk/news/warnings/24fxoddpro


In [17]:
bs_list = []


for rec in frame_:
    items = {}
    
    
    name_ = rec['Name']

    country = rec["Country"].strip()

    if country:
        country = [country]
    else:
        country = []
    
    
    
    items['uid'] = get_hash(name_)
    items['name'] = name_.strip()
    items['alias'] = [rec['Alias']] if rec['Alias'].strip() else []
    items['country'] = country
    items["list_type"] = "Entity"
    items['last_updated'] = last_updated_string
    items['entity_details'] = {}
    items['entity_details']['phone'] = [rec['Contact Number']]
    items['entity_details']['email'] = [rec['Email']]
    items['entity_details']['website'] =[ rec['Website']]
    items['documents'] = {}
    items['sanction_details'] = {}
    items['sanction_details']['body'] = rec['Firm Type']
    items['nns_status'] = "False"
    items['address'] =[ {
                            "complete_address": str(rec['Complete_address']).strip(),
                            "state": rec['State'].strip(),
                            "city": rec['City'].strip(),
                            "zip_code": str(rec['Pin code']),
                            "country" : rec['Country'].strip()
        
    }]
    items['comment'] = ""
    items['sanction_list'] = {}
    items['sanction_list']['sl_authority'] = "Financial Conduct Authority, United Kingdom"
    items['sanction_list']['sl_url'] = "https://www.fca.org.uk/"
    items['sanction_list']['watch_list'] = "European Watchlists"
    items['sanction_list']['sl_type'] = "Sanctions"
    items['sanction_list']['sl_host_country'] = "United Kingdom"
    items['sanction_list']['sl_source'] = "FCA (UK) Unauthorised firms and Individuals List, United Kingdom"
    items['sanction_list']['sl_description'] = "List of Clone and Unauthorised firms by Financial Conduct Authority, United Kingdom"
    items['sanction_list']['list_id'] = "UK_E20019"
    bs_list.append(items)
    

In [18]:
bs_list[0]

{'uid': '611aee3144056b3f909f0c9bb93cd54730e4aade85429cd6f55929bc1672f54f',
 'name': '24FOREX TRADE STOCKS',
 'alias': [],
 'country': ['United Kingdom'],
 'list_type': 'Entity',
 'last_updated': '2022-05-19T07:36:42',
 'entity_details': {'phone': [17178236772],
  'email': ['support@24forextradestocks.com'],
  'website': ['https://24forextradestocks.com/']},
 'documents': {},
 'sanction_details': {'body': 'Unauthorised'},
 'nns_status': 'False',
 'address': [{'complete_address': '66 Great Suffolk Street',
   'state': 'England',
   'city': 'London',
   'zip_code': 'SE1 0BL',
   'country': 'United Kingdom'}],
 'comment': '',
 'sanction_list': {'sl_authority': 'Financial Conduct Authority, United Kingdom',
  'sl_url': 'https://www.fca.org.uk/',
  'watch_list': 'European Watchlists',
  'sl_type': 'Sanctions',
  'sl_host_country': 'United Kingdom',
  'sl_source': 'FCA (UK) Unauthorised firms and Individuals List, United Kingdom',
  'sl_description': 'List of Clone and Unauthorised firms by 

In [19]:
with open('SNG-508_Entity.json', 'w', encoding="utf-8") as file:
    json.dump(bs_list, file, ensure_ascii=False, indent=2, default=str)